In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,45.03,89,80,30.94,TF,1708405920
1,1,adamstown,-25.0660,-130.1015,78.13,86,75,10.83,PN,1708405924
2,2,kuchaiburi,22.2667,86.1667,89.26,24,10,10.38,IN,1708406429
3,3,rio verde de mato grosso,-18.9181,-54.8442,72.70,94,100,1.23,BR,1708406430
4,4,edinburgh of the seven seas,-37.0676,-12.3116,59.50,64,43,9.69,SH,1708405935


In [29]:
import pandas as pd
import hvplot.pandas

# Create a scatter plot with 'Lat' and 'Lng' as x and y coordinates, and 'Humidity' as the size of the points
world_map_plot = city_data_df.hvplot.scatter(x='Lng', y='Lat', size='Humidity', geo=True,
                                             hover_cols=['City', 'Country', 'Humidity'],
                                             title='City Humidity on World Map',
                                             xlabel='Longitude', ylabel='Latitude',
                                             xlim=(-180, 180), ylim=(-90, 90))

# Display the scatter plot on a world map
world_map_plot


:Scatter   [Lng]   (Lat,Humidity,City,Country)

In [30]:
# Narrow down cities that fit criteria (e.g., maximum temperature < 80°F, humidity < 80%, cloudiness = 0)
criteria_df = city_data_df[(city_data_df['Max Temp'] < 60) & 
                           (city_data_df['Humidity'] < 80) & 
                           (city_data_df['Wind Speed'] < 6) &
                           (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
criteria_df = criteria_df.dropna()

# Display sample data
print(criteria_df)


     City_ID             City      Lat       Lng  Max Temp  Humidity  \
12        12          stanley  54.8680   -1.6985     44.78        79   
16        16           albany  42.6001  -73.9662     19.31        63   
90        90           debila  33.5167    6.9500     51.87        71   
95        95           tlaxco  19.6167  -98.1167     50.09        34   
193      193       san julian  21.0167 -102.1667     59.40        20   
220      220         hannibal  39.7084  -91.3585     43.72        42   
224      224      al burayqah  30.4062   19.5739     56.07        66   
256      256          gazojak  41.1875   61.4036     25.05        45   
261      261            darab  28.7519   54.5444     51.82        32   
262      262           rikaze  29.2500   88.8833     39.38        23   
271      271           bechar  31.6167   -2.2167     46.72        46   
279      279           waddan  29.1614   16.1390     50.72        76   
289      289  masjed soleyman  31.9364   49.3039     58.42      

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = criteria_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df.head())


           City Country      Lat       Lng  Humidity Hotel Name
12      stanley      GB  54.8680   -1.6985        79           
16       albany      US  42.6001  -73.9662        63           
90       debila      DZ  33.5167    6.9500        71           
95       tlaxco      MX  19.6167  -98.1167        34           
193  san julian      MX  21.0167 -102.1667        20           


In [32]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
params = {
    "radius": radius,
    "type": "hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}"
    params["bias"] = f"point:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
stanley - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
debila - nearest hotel: No hotel found
tlaxco - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
hannibal - nearest hotel: No hotel found
al burayqah - nearest hotel: No hotel found
gazojak - nearest hotel: No hotel found
darab - nearest hotel: No hotel found
rikaze - nearest hotel: No hotel found
bechar - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
masjed soleyman - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
gazli - nearest hotel: No hotel found
la junta - nearest hotel: No hotel found
colby - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
fort knox - nearest hotel: No hotel found
tubarjal - nearest hotel: No hotel found
tindouf - nearest hotel: No hotel found
sterling - nearest hotel: No hotel found
la ronge - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
12,stanley,GB,54.8680,-1.6985,79,No hotel found
16,albany,US,42.6001,-73.9662,63,No hotel found
90,debila,DZ,33.5167,6.9500,71,No hotel found
95,tlaxco,MX,19.6167,-98.1167,34,No hotel found
193,san julian,MX,21.0167,-102.1667,20,No hotel found
220,hannibal,US,39.7084,-91.3585,42,No hotel found
224,al burayqah,LY,30.4062,19.5739,66,No hotel found
256,gazojak,TM,41.1875,61.4036,45,No hotel found
261,darab,IR,28.7519,54.5444,32,No hotel found
262,rikaze,CN,29.2500,88.8833,23,No hotel found


In [33]:
# Create a scatter plot with 'Lat' and 'Lng' as x and y coordinates, and 'Humidity' as the size of the points
world_map_plot = hotel_df.hvplot.scatter(x='Lng', y='Lat', size='Humidity', geo=True,
                                         hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],
                                         title='City Humidity with Nearest Hotel',
                                         xlabel='Longitude', ylabel='Latitude',
                                         xlim=(-180, 180), ylim=(-90, 90))

# Display the scatter plot on a world map
world_map_plot


:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)